### YOLOv8을 패키지로 설치하기

In [1]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 KB 9.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 34.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 KB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 139.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 32.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 86.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 KB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4

#### 엘리스 코드에서는 libgl1-mesa-glx 라이브러리가 기본으로 설치되어 있지 않아 추가 설치 필요

In [4]:
!sudo apt-get update && apt-get -y install libgl1-mesa-glx -y

Hit:1 http://mirror.elice.io/ubuntu jammy InRelease
Hit:2 http://mirror.elice.io/ubuntu jammy-updates InRelease                    
Hit:3 http://mirror.elice.io/ubuntu jammy-backports InRelease                  
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease             
Reading package lists... Done
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


#### YOLOv8이 정상적으로 설치되었는지 확인하기 위해 predict()실행
아래 predict()는 coco데이터셋(80개 클래스)으로 학습된 가중치 파일을 사용

In [5]:
!yolo task=detect mode=predict model=yolov8s.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

Ultralytics YOLOv8.0.121 🚀 Python-3.10.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe MIG 1g.10gb, 9728MiB)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients

Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 /home/elicer/dog.jpeg: 640x384 1 person, 1 car, 1 dog, 1 handbag, 57.4ms
Speed: 1.4ms preprocess, 57.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict2


### 작업 폴더를 생성

In [1]:
import os

dataPath = '/home/elicer/drone/'

os.makedirs(dataPath, exist_ok=True)
os.chdir(dataPath)

#### 데이터셋 압축파일 풀기

In [ ]:
!unzip /home/elicer/drone/KSAdataset.zip -d /home/elicer/drone

#### YOLOv8 학습하기
학습을 중단했다가 다시 resume으로 이어하려면 epoch은 넉넉하게 설정해주자. epoch을 500으로 설정한뒤 100에서 중단한 다음 resume옵션으로 이어서 학습 가능 

In [9]:
!yolo task=detect mode=train model=yolov8s.pt data=/home/elicer/drone/data.yaml epochs=500 imgsz=640 plots=True

Ultralytics YOLOv8.0.121 🚀 Python-3.10.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe MIG 1g.10gb, 9728MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/elicer/drone/data.yaml, epochs=500, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=

#### 학습을 잠시 중단하고 얼마나 잘 인식하는지 테스트
predict시에 여러 장을 한꺼번에 실행하려면 리스트를 저장하고 있는 텍스트 파일을 만들어줍니다.

In [2]:
import os
#이미지가 저장된 경로
imagePath = '/home/elicer/drone/test/images'
testList = os.listdir(imagePath)

txtfile = open("testlist.txt", 'w')

for filename in testList:
    txtfile.write('/home/elicer/drone/test/images/'+filename+'\n')

txtfile.close()

#### 생성된 파일을 확인

In [6]:
!cat testlist.txt 

/home/elicer/drone/test/images/0509_V0028_136.jpg
/home/elicer/drone/test/images/0509_V0028_207.jpg
/home/elicer/drone/test/images/10579761_00007.jpg
/home/elicer/drone/test/images/1572547_00006.jpg
/home/elicer/drone/test/images/3945877_00002.jpg
/home/elicer/drone/test/images/4185375_00005.jpg
/home/elicer/drone/test/images/4198787_00007.jpg
/home/elicer/drone/test/images/6-FLY-AI05-SUN-5M-15-LAND-Goseong-0116-13-33-02-L-N_0001749.jpg
/home/elicer/drone/test/images/6-FLY-AI05-SUN-5M-15-LAND-Goseong-0117-10-41-02-L-N_0002440.jpg
/home/elicer/drone/test/images/6-FLY-AI05-SUN-5M-15-LAND-Goseong-0118-12-06-05-L-Y_0000403.jpg
/home/elicer/drone/test/images/6-FLY-AI05-SUN-5M-15-LAND-Gyeongju-0119-13-18-21-L-N_0002264.jpg
/home/elicer/drone/test/images/6-FLY-AI05-SUN-5M-15-LAND-Gyeongju-0119-14-01-07-L-N_0002218.jpg
/home/elicer/drone/test/images/8101288_00006.jpg
/home/elicer/drone/test/images/9310414_00005.jpg
/home/elicer/drone/test/images/9950837_00005.jpg
/home/elicer/drone/test/images

#### 생성된 테스트 리스트를 사용하여 predict를 실행합니다. 
##### 결과파일은 runs/detect/predict/아래에 저장됩니다.

In [7]:
!yolo task=detect mode=predict model=/home/elicer/drone/runs/detect/train3/weights/best.pt conf=0.25 source=/home/elicer/drone/testlist.txt

Ultralytics YOLOv8.0.121 🚀 Python-3.10.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe MIG 1g.10gb, 9728MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients

image 1/104 /home/elicer/drone/test/images/0509_V0028_136.jpg: 384x640 3 persons, 2 faces, 2 cars, 57.9ms
image 2/104 /home/elicer/drone/test/images/0509_V0028_207.jpg: 384x640 3 persons, 1 face, 1 car, 6.0ms
image 3/104 /home/elicer/drone/test/images/10579761_00007.jpg: 384x640 2 persons, 1 face, 6.0ms
image 4/104 /home/elicer/drone/test/images/1572547_00006.jpg: 384x640 2 persons, 1 car, 1 number plate, 7.0ms
image 5/104 /home/elicer/drone/test/images/3945877_00002.jpg: 384x640 2 persons, 2 faces, 6.6ms
image 6/104 /home/elicer/drone/test/images/4185375_00005.jpg: 640x352 1 person, 75.0ms
image 7/104 /home/elicer/drone/test/images/4198787_00007.jpg: 384x640 (no detections), 6.4ms
image 8/104 /home/elicer/drone/test/images/6-FLY-AI05-SUN-5M-15-LAND-Goseong-0116-13-33-02-L-N_0001749.jpg: 384x640 1 person, 1 c

### 중단된 학습 이어서 하기(resume)

In [20]:
# 학습 이어서 하기 (resume)
!yolo train resume=True model=/home/elicer/drone/runs/detect/train3/weights/last.pt

WARNING ⚠️ 'data' is missing. Using default 'data=coco128.yaml'.
Ultralytics YOLOv8.0.121 🚀 Python-3.10.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe MIG 1g.10gb, 9728MiB)
yolo/engine/trainer: task=detect, mode=train, model=/home/elicer/drone/runs/detect/train3/weights/last.pt, data=/home/elicer/drone/data.yaml, epochs=500, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, 

### mAP를 높여봅시다.


##### 인식율이 낮게 나오는데 가장 크게 영향을 끼치는 클래스가 자동차 번호판입니다.  